In [ ]:
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
compartments = (
    "susceptible", 
    "latent_early",
    "latent_late",
    "infectious",
    "treatment", 
    "recovered",
)
tb_model = CompartmentalModel(
    times=[1800, 2024],
    compartments=compartments,
    infectious_compartments=["infectious"],
)
start_pop = {
    "susceptible": 4e7,
    "infectious": 1.0,
}
tb_model.set_initial_population(start_pop)

# Demographics will be important
tb_model.add_crude_birth_flow(
    "birth",
    Parameter("crude_birth_rate"),
    "susceptible",
)
tb_model.add_universal_death_flows(
    "all_cause_mortality",
    1.0 / Parameter("life_expectancy"),
)
tb_model.request_output_for_compartments("total_population", compartments)

In [ ]:
parameters = {
    "crude_birth_rate": 20.0 / 1000.0,
    "life_expectancy": 60.0,
}
tb_model.run(parameters)

In [ ]:
tb_model.get_derived_outputs_df().plot()